# DUBLIN CITY COUNCIL AI ASSISTANT - COMPLETE WORKFLOW


## 1: INSTALL DEPENDENCIES (~2 minutes)

In [1]:
print("📦 Installing all dependencies...")
!pip install -q --upgrade \
    "numpy>=2.0,<2.3" \
    "scipy>=1.13.0" \
    "transformers" \
    "sentence-transformers" \
    "faiss-cpu" \
    "langchain" \
    "langchain-community" \
    "crewai" \
    "crewai-tools" \
    "beautifulsoup4" \
    "requests" \
    "unstructured"

print("✅ Dependencies installed!")

📦 Installing all dependencies...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.4/80.4 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 66.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 125.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 107.3 MB/s et

In [4]:
! pip install LiteLLM

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.1/278.1 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 112.2 MB/s eta 0:00:00
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.76.0
    Uninstalling grpcio-1.76.0:
      Successfully uninstalled grpcio-1.76.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.2 requires grpcio>=1.71.2, but you have grpcio 1.67.1 which is incompatible.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.2.6 which is incompatible.


## 2: SETUP OLLAMA + PHI-3 (~3 minutes)


In [2]:
import subprocess, time

print("🚀 Setting up Ollama and Phi-3...")
!curl -fsSL https://ollama.com/install.sh | sh
subprocess.Popen(['ollama', 'serve'], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
time.sleep(5)
!ollama pull phi3

print("✅ Ollama + Phi-3 ready!")

🚀 Setting up Ollama and Phi-3...
>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.

✅ Ollama + Phi-3 ready!


## 3: SCRAPE DUBLIN CITY COUNCIL PAGES (~1 minute)

In [3]:
import requests
from bs4 import BeautifulSoup
import os

os.makedirs("/content/data/raw", exist_ok=True)

def scrape_page(url, name):
    try:
        print(f"  📥 {name}...")
        res = requests.get(url, timeout=10)
        soup = BeautifulSoup(res.text, 'html.parser')
        for script in soup(["script", "style", "nav", "footer"]):
            script.decompose()
        text = soup.get_text(separator="\n", strip=True)
        with open(f"/content/data/raw/{name}.txt", "w", encoding="utf-8") as f:
            f.write(f"Source: {url}\n\n{text}")
        print(f"  ✅ {len(text):,} chars")
        return True
    except Exception as e:
        print(f"  ❌ {e}")
        return False

pages = {
    "waste_recycling": "https://www.dublincity.ie/residential/waste-and-recycling",
    "housing_support": "https://www.dublincity.ie/residential/housing",
    "parking_permits": "https://www.dublincity.ie/residential/parking",
    "planning_permission": "https://www.dublincity.ie/residential/planning",
    "community_services": "https://www.dublincity.ie/residential/community",
    "libraries": "https://www.dublincity.ie/residential/libraries",
    "parks_recreation": "https://www.dublincity.ie/residential/parks-and-recreation",
    "social_housing": "https://www.dublincity.ie/residential/housing/social-housing",
    "housing_grants": "https://www.dublincity.ie/residential/housing/housing-grants",
    "bin_collection": "https://www.dublincity.ie/residential/waste-and-recycling/bin-collection",
}

print(f"🌐 Scraping {len(pages)} pages...\n")
success = sum(scrape_page(url, name) for name, url in pages.items())
print(f"\n✅ Scraped {success}/{len(pages)} pages!")


🌐 Scraping 10 pages...

  📥 waste_recycling...
  ✅ 197 chars
  📥 housing_support...
  ✅ 354 chars
  📥 parking_permits...
  ✅ 197 chars
  📥 planning_permission...
  ✅ 347 chars
  📥 community_services...
  ✅ 197 chars
  📥 libraries...
  ✅ 3,329 chars
  📥 parks_recreation...
  ✅ 197 chars
  📥 social_housing...
  ✅ 197 chars
  📥 housing_grants...
  ✅ 197 chars
  📥 bin_collection...
  ✅ 197 chars

✅ Scraped 10/10 pages!


## 4: BUILD FAISS INDEX (~1 minute)


In [4]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

print("🔨 Building FAISS index...")

# Load documents
loader = DirectoryLoader("/content/data/raw", glob="**/*.txt", loader_cls=TextLoader)
docs = loader.load()
print(f"  ✅ Loaded {len(docs)} documents")

# Split into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)
print(f"  ✅ Created {len(chunks)} chunks")

# Create embeddings and index
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cuda'}
)
vectorstore = FAISS.from_documents(chunks, embeddings)
vectorstore.save_local("/content/data/faiss_index")
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

print("✅ FAISS index ready!")


🔨 Building FAISS index...
  ✅ Loaded 10 documents
  ✅ Created 19 chunks


/tmp/ipython-input-2926891097.py:19: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


✅ FAISS index ready!


## 5: CREATE RAG TOOL

In [7]:
from crewai.tools import BaseTool
from pydantic import BaseModel, Field

class RAGSearchInput(BaseModel):
    query: str = Field(..., description="Search query")

class DublinCouncilRAGTool(BaseTool):
    name: str = "DublinCouncilPolicySearch"
    description: str = "Search Dublin City Council policies and information"
    args_schema: type[BaseModel] = RAGSearchInput

    def _run(self, query: str) -> str:
        try:
            docs = retriever.invoke(query)
            if not docs: return "No documents found."
            excerpts = [f"[Doc {i+1}]: {d.page_content[:300]}" for i, d in enumerate(docs[:3])]
            return "\n\n".join(excerpts)
        except Exception as e:
            return f"Error: {e}"

rag_tool = DublinCouncilRAGTool()
print("✅ RAG tool created!")

✅ RAG tool created!


## 6: CREATE AI AGENTS


In [8]:
from crewai import LLM, Agent

llm = LLM(model="ollama/phi3", base_url="http://localhost:11434", temperature=0.1)

researcher = Agent(
    role="Policy Researcher",
    goal="Find accurate Dublin City Council information",
    backstory="Expert at searching official documents. Always uses the search tool.",
    llm=llm, tools=[rag_tool], verbose=True, max_iter=3, allow_delegation=False
)

validator = Agent(
    role="Eligibility Validator",
    goal="Determine YES/NO eligibility",
    backstory="Strict policy interpreter. Makes decisions based only on research.",
    llm=llm, verbose=True, allow_delegation=False
)

actioner = Agent(
    role="Citizen Support Advisor",
    goal="Provide clear next steps",
    backstory="Helps citizens navigate bureaucracy with empathy and clarity.",
    llm=llm, verbose=True, allow_delegation=False
)

print("✅ All agents created!")

✅ All agents created!


## 7: CREATE WORKFLOW FUNCTION

In [9]:
from crewai import Task, Crew, Process

def run_query(user_query: str):
    """Run multi-agent workflow"""
    task1 = Task(
        description=f"Search for: '{user_query}'. Use the DublinCouncilPolicySearch tool.",
        expected_output="Relevant policy information",
        agent=researcher
    )
    task2 = Task(
        description="Determine eligibility. Answer YES or NO with reasoning.",
        expected_output="Clear eligibility determination",
        agent=validator, context=[task1]
    )
    task3 = Task(
        description="Provide specific next steps and contact information.",
        expected_output="Actionable guidance",
        agent=actioner, context=[task1, task2]
    )

    crew = Crew(
        agents=[researcher, validator, actioner],
        tasks=[task1, task2, task3],
        process=Process.sequential,
        verbose=True
    )

    return crew.kickoff()

print("✅ Workflow ready!")


✅ Workflow ready!


## 8: TEST THE SYSTEM


In [10]:
print("\n" + "="*70)
print("🧪 TESTING SYSTEM")
print("="*70)

test_query = "I'm a student in Dublin 6W. Can I apply for a bin collection exemption?"
print(f"\nQuery: {test_query}\n")

result = run_query(test_query)

print("\n" + "="*70)
print("🤖 FINAL RESPONSE:")
print("="*70)
print(result.raw if hasattr(result, 'raw') else result)


🧪 TESTING SYSTEM

Query: I'm a student in Dublin 6W. Can I apply for a bin collection exemption?



╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 10d2f8f7-9b0b-47a7-afd2-0e081381e1af                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Policy Researcher                                                                                       │
│                                                                                                                 │
│  Task: Search for: 'I'm a student in Dublin 6W. Can I apply for a bin collection exemption?'. Use the           │
│  DublinCouncilPolicySearch tool.                                                                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

Output()

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Citizen Support Advisor                                                                                 │
│                                                                                                                 │
│  Task: Provide specific next steps and contact information.                                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: bd6979dc-419e-4f46-b064-b3c576cf1035                                                                     │
│  Agent: Eligibility Validator                                                                                   │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Citizen Support Advisor                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  To obtain accurate information about a potential 'bin collection exemption for students in Dublin 6W', please  │
│  contact the Dublin City Council directly. You can reach them by phone at +353 XXX-XXXX or via email at         │
│  info@dublincitycouncil.ie. Additionally, visit their official website to check if there are any updates on     │
│  this matter: https://www.dublincitycouncil.ie/. If you have a specific reason for the exemption request that   │
│  may qualify under other policies or special circumstances not covered in your search results, be sure to       │
│  mention it when contacting them so they can provide tailored advice and guidance.                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

## 9: INTERACTIVE MODE (OPTIONAL)


In [ ]:
def interactive_mode():
    """Run interactive Q&A"""
    print("\n" + "="*70)
    print("🏛️  DUBLIN CITY COUNCIL AI ASSISTANT")
    print("="*70)
    print("Type your questions. Type 'exit' to quit.\n")

    while True:
        query = input("📝 Your Question: ").strip()
        if query.lower() in ['exit', 'quit', 'q']:
            print("👋 Goodbye!")
            break
        if not query:
            continue

        print("\n🔄 Processing...\n")
        try:
            result = run_query(query)
            response = result.raw if hasattr(result, 'raw') else str(result)
            print("\n" + "="*70)
            print("🤖 RESPONSE:")
            print("="*70)
            print(response)
            print("="*70 + "\n")
        except Exception as e:
            print(f"❌ Error: {e}\n")

# Uncomment to start interactive mode:
interactive_mode()

# ============================================================================
# EXAMPLE QUERIES TO TRY
# ============================================================================
"""
Try these example queries:

1. "What are the requirements for planning permission in Dublin?"
2. "How do I apply for a parking permit in the city center?"
3. "Can I get a housing grant as a first-time buyer?"
4. "What waste collection services are available in my area?"
5. "Are there community services for elderly residents?"
6. "How do I report illegal dumping?"
7. "What documents do I need for social housing application?"
8. "Where are the recycling centres in Dublin?"
9. "How much are parking permits for residents?"
10. "What grants are available for home improvements?"
"""

print("\n✅ System ready! Use run_query() or interactive_mode() to ask questions.")


🏛️  DUBLIN CITY COUNCIL AI ASSISTANT
Type your questions. Type 'exit' to quit.

📝 Your Question: How can I get the bills of gas and electricity on my name? What are the steps for it?

🔄 Processing...



╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 03ca1222-d3f1-4150-a942-bdc4a963bb59                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Policy Researcher                                                                                       │
│                                                                                                                 │
│  Using Tool: DublinCouncilPolicySearch                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Citizen Support Advisor                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  To address your query concerning getting bills for gas and electricity under individual names as per official  │
│  documentation from the Dublin City Council (DCC), it appears that there is no direct policy available in       │
│  Document [Doc 1] or general information on DCC’s website regarding this specific eligibility. However, since   │
│  utility connections are often part of housing services provided by DCC and given your interest lies within a   │
│  residential context as suggested by the mentioning of Housing policies (Document [Doc 2]), I recommend         │
│  contacting Dublin City Council's Housing Services directly for personalized assistance on this matter. You     │
│  can reach them at:                                                                                             │
│                                                                                                                 │
│  Dublin City Council, Customer Service Centre                                                                   │
│  Phone Number: +353 XXX XXXX XXXXX or email: housing@dubcitycouncil.ie                                          │
│                                                                                                                 │
│  Additionally, while Document [Doc 3] is not directly accessible for review in this task scenario and does not  │
│  provide specific guidance on billing issues related to waste management services that could be linked with     │
│  utilities like gas and electricity meters, it may still contain relevant information regarding general         │
│  utility-related queries. Therefore, I advise visiting the Dublin City Council's official website or            │
│  contacting their Waste & Recycling Services for further clarification on this aspect:                          │
│  Dublin City Council, Customer Service Centre (Waste & Recycling)                                               │
│  Phone Number: +353 XXX XXXX XXXXX or email: waste@dubcitycouncil.ie                                            │
│                                                                                                                 │
│  Please note that these steps are based on the assumption of a housing-related context for your query and may   │
│  not directly answer individual bill eligibility status, but they should guide you towards obtaining more       │
│  specific information from DCC's Housing Services or Waste & Recycling services as appropriate.                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()


🤖 RESPONSE:
To address your query concerning getting bills for gas and electricity under individual names as per official documentation from the Dublin City Council (DCC), it appears that there is no direct policy available in Document [Doc 1] or general information on DCC’s website regarding this specific eligibility. However, since utility connections are often part of housing services provided by DCC and given your interest lies within a residential context as suggested by the mentioning of Housing policies (Document [Doc 2]), I recommend contacting Dublin City Council's Housing Services directly for personalized assistance on this matter. You can reach them at:

Dublin City Council, Customer Service Centre
Phone Number: +353 XXX XXXX XXXXX or email: housing@dubcitycouncil.ie

Additionally, while Document [Doc 3] is not directly accessible for review in this task scenario and does not provide specific guidance on billing issues related to waste management services that could be lin

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 4011b913-58f9-4213-a4d6-bace79d8c9ea                                                                     │
│  Agent: Citizen Support Advisor                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

exit
📝 Your Question: exit
